In [4]:
from absl import app, flags, logging
from absl.flags import FLAGS
import cv2
import os
import numpy as np
import tensorflow as tf
from glob import glob

from evaluations import get_val_data, perform_val
from models import ArcFaceModel
from utils import set_memory_growth, load_yaml

In [5]:
### IJB-C Dataset

# batch_size = 128
# input_size = 112
# embd_shape = 512
# head_type = 'ArcHead'
# backbone_type = 'MobileNetV2'
# w_decay=5e-4
# num_classes = 3584 
# base_lr = 0.01
# dataset_len = 13033 
# epochs = 100
# steps_per_epoch = dataset_len // batch_size

### MS1M dataset

batch_size = 128 # Initially 128
input_size = 112
embd_shape = 512
head_type = 'CosHead' # ''ArcHead', CosHead'
# Backbones w/ pretrained weights:
#     MobileNet, MobileNetV2, InceptionResNetV2, InceptionV3, ResNet50, ResNet50V2, ResNet101V2, NASNetLarge, NASNetMobile, Xception
#     But if you are trying to use NasNet, please check this issue first: https://github.com/keras-team/keras-applications/issues/78
#         We manually download the weight file and explicitly load it in models.py file
# Backbones w/o pretrained weights:
#     MobileNetV3Large, MobileNetV3Small, EfficientNetLite0~6, EfficientNetB0~7
backbone_type = 'EfficientNetLite0' 
w_decay=5e-4
num_classes = 85742 
dataset_len = 5822653 
base_lr = 0.01 # initially 0.01
epochs = 20
save_steps = 1000
train_size = int(0.8 * dataset_len)
print(train_size)
steps_per_epoch = train_size // batch_size
print(steps_per_epoch)
val_size = dataset_len - train_size
print(val_size)
validation_steps = val_size // batch_size
print(validation_steps)
steps = 1
is_ccrop=False
binary_img=True
is_Adam = False

dgx = True

4658122
36391
1164531
9097


In [9]:
if dgx:
    base_dir = "/raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/"
    # save_name = "ms1m_mobilenet_check/SGD/*"
    save_name = "ms1m_"+backbone_type+"_"+head_type+"_check/"+version+"/*"
else:
    base_dir = "/hd/honghee/models/checkpoints/w_tfidentity/"
    save_name = "ms1m_"+backbone_type+"_"+head_type+"_check/"+version+"/*"
    
## collect loss in checkpoints
file_list = []
for files in glob(base_dir+save_name):
#     file_list.append(files.split('/')[-1].split('l_')[-1])
    if not files.split('/')[-1].split('l_')[-1].split('.ckpt')[0] == 'checkpoint':
        loss = float( files.split('/')[-1].split('l_')[-1].split('.ckpt')[0] )
    file_list.append( loss  )
file_list.sort()

# print(file_list[0:3])

load_file_name = []
for files in glob(base_dir+save_name):
    if files.split('/')[-1].split('l_')[-1].split('.ckpt')[0] == 'checkpoint':
        pass
    elif file_list[0] == float( files.split('/')[-1].split('l_')[-1].split('.ckpt')[0] ) and files.split('/')[-1].split('l_')[-1].split('.ckpt')[-1]!='.index':
        load_file_name = files
best_checkpoint = load_file_name.split('.data')[0]
initial_epoch = int(load_file_name.split('e_')[-1].split('_')[0])-1
print(initial_epoch)
print(best_checkpoint)

0
/raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_CosHead_check/SGD/e_1_l_47.875667572021484.ckpt


In [10]:
# flags.DEFINE_string('cfg_path', './configs/arc_res50.yaml', 'config file path')
# flags.DEFINE_string('gpu', '0', 'which gpu to use')
# flags.DEFINE_string('img_path', '', 'path to input image')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

logger = tf.get_logger()
logger.disabled = True
logger.setLevel(logging.FATAL)
set_memory_growth()

# cfg = load_yaml(FLAGS.cfg_path)

model = ArcFaceModel(size=input_size,
                     backbone_type=backbone_type,
                     training=False)

model.load_weights(best_checkpoint)

# ckpt_path = tf.train.latest_checkpoint('./checkpoints/' + cfg['sub_name'])
# if ckpt_path is not None:
#     print("[*] load ckpt from {}".format(ckpt_path))
#     model.load_weights(ckpt_path)
# else:
#     print("[*] Cannot find ckpt from {}.".format(ckpt_path))
#     exit()

# if FLAGS.img_path:
#     print("[*] Encode {} to ./output_embeds.npy".format(FLAGS.img_path))
#     img = cv2.imread(FLAGS.img_path)
#     img = cv2.resize(img, (cfg['input_size'], cfg['input_size']))
#     img = img.astype(np.float32) / 255.
#     if len(img.shape) == 3:
#         img = np.expand_dims(img, 0)
#     embeds = l2_norm(model(img))
#     np.save('./output_embeds.npy', embeds)

print("[*] Loading LFW, AgeDB30 and CFP-FP...")
lfw, agedb_30, cfp_fp, lfw_issame, agedb_30_issame, cfp_fp_issame = get_val_data("/raid/workspace/honghee/")

print("[*] Perform Evaluation on LFW...")
acc_lfw, best_th = perform_val(
    embd_shape, batch_size, model, lfw, lfw_issame,
    is_ccrop=False)
print("    acc {:.4f}, th: {:.2f}".format(acc_lfw, best_th))

print("[*] Perform Evaluation on AgeDB30...")
acc_agedb30, best_th = perform_val(
    embd_shape, batch_size, model, agedb_30,
    agedb_30_issame, is_ccrop=False)
print("    acc {:.4f}, th: {:.2f}".format(acc_agedb30, best_th))

print("[*] Perform Evaluation on CFP-FP...")
acc_cfp_fp, best_th = perform_val(
    embd_shape, batch_size, model, cfp_fp, cfp_fp_issame,
    is_ccrop=False)
print("    acc {:.4f}, th: {:.2f}".format(acc_cfp_fp, best_th))

[*] Loading LFW, AgeDB30 and CFP-FP...


FileNotFoundError: [Errno 2] No such file or directory: '/raid/workspace/honghee/lfw_align_112/lfw/meta/sizes'